Cours : SQL pour Débutant(e)s Jour 2 - Fondamentaux des Requêtes SQL
Lien : https://www.youtube.com/watch?v=bpRirgAiyhw&t=108s

Github le coin stat : https://github.com/LeCoinStat/LeCoinStat/tree/main/Bien_Debuter_SQL
Fichiers Github récupérés : https://github.com/LeCoinStat/LeCoinStat/blob/main/Bien_Debuter_SQL/Jour1/CreationTable

In [3]:
import duckdb
import polars as pl
from skimpy import skim

In [4]:
# Création d'une BD virtuelle
conn = duckdb.connect()

## Fichier des produits

In [12]:
# Requête SQL : récupération de toutes les données du fichier .csv en DF
products_df = conn.execute(
    """
    SELECT *
    FROM read_csv_auto('data/indexE/Produits.csv', header=True)
    """
).df()

In [13]:
# Données stat
skim(products_df)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 100    │ │ int32       │ 3     │                                                          │
│ │ Number of columns │ 5      │ │ string      │ 2     │                                                          │
│ └───────────────────┴────────┘ └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━┓  │
│ ┃ column_name        ┃ NA   ┃ NA %   ┃ mean    ┃ sd      ┃ p0  ┃ p25     ┃ p50    ┃ p75    ┃ p100  ┃ hist    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━┩  │
│ │ ProduitID          │    0 │      0 │    50.5 │   29.01 │   1 │   25.75 │   50.5 │  75.25 │   100 │ ▇▇▇▇▇▇  │  │
│ │ PrixUnitaire       │    0 │      0 │   486.7 │   326.6 │  59 │     129 │    399 │    699 │   999 │ ▇▃▂ ▇▃  │  │
│ │ FournisseurID      │    0 │      0 │   54.04 │   30.51 │   1 │      31 │     56 │     84 │   100 │ ▆▃▅▃▅▇  │  │
│ └────────────────────┴──────┴────────┴─────────┴─────────┴─────┴─────────┴────────┴────────┴───────┴─────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name               ┃ NA      ┃ NA %       ┃ words per row                ┃ total words              ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ NomProduit                │       0 │          0 │                          2.7 │                      266 │  │
│ │ Description               │       0 │          0 │                           14 │                     1371 │  │
│ └───────────────────────────┴─────────┴────────────┴──────────────────────────────┴──────────────────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

In [14]:
# Enregistrement de la DF ci-avant en BD virtuelle (SQL : CTE)
conn.register('products_df', products_df)

## Requêtes SQL

Liste des produits vendus > à 200 €

In [15]:
conn.execute(
    """
    -- Récupération des champs
    SELECT
        "NomProduit" AS "Nom du produit"
        , "PrixUnitaire" AS "Prix unitaire"
    FROM products_df
    -- Filtre
    WHERE "Prix unitaire" > 200
    """
).df()

,Nom du produit,Prix unitaire
0,Samsung Galaxy S21,699
1,Levi's Jeans,299
2,Dyson Vacuum,999
3,Dell XPS 13,299
4,Nike Air Max,799
...,...,...
65,iPhone 12,699
66,iPhone 12,999
67,Dell XPS 13,799
68,Dyson Vacuum,999


Information sur le produit "Nike Air Max"

In [16]:
conn.execute(
    """
    -- Champs à récupérer
    SELECT *
    FROM products_df
    -- Filtre
    WHERE "NomProduit" LIKE 'Nike Air Max'
    """
).df()

,ProduitID,NomProduit,Description,PrixUnitaire,FournisseurID
0,3,Nike Air Max,"Running shoes with Max Air cushioning, mesh up...",85,93
1,8,Nike Air Max,"Running shoes with Max Air cushioning, mesh up...",799,90
2,13,Nike Air Max,"Running shoes with Max Air cushioning, mesh up...",59,53
3,18,Nike Air Max,"Running shoes with Max Air cushioning, mesh up...",699,34
4,23,Nike Air Max,"Running shoes with Max Air cushioning, mesh up...",699,18
5,26,Nike Air Max,"Running shoes with Max Air cushioning, mesh up...",85,52
6,39,Nike Air Max,"Running shoes with Max Air cushioning, mesh up...",299,35
7,58,Nike Air Max,"Running shoes with Max Air cushioning, mesh up...",399,79
8,63,Nike Air Max,"Running shoes with Max Air cushioning, mesh up...",399,33
9,70,Nike Air Max,"Running shoes with Max Air cushioning, mesh up...",799,98


Information sur les produits du fournisseur n° 13

In [17]:
conn.execute(
    """
    -- Champs à récupérer
    SELECT *
    FROM products_df
    -- Filtre
    WHERE "FournisseurID" = 13
    """
).df()

,ProduitID,NomProduit,Description,PrixUnitaire,FournisseurID
0,19,Dyson Vacuum,Cordless vacuum cleaner with strong suction po...,59,13
1,41,Dell XPS 13,13.4-inch FHD+ laptop with 10th Gen Intel Core...,299,13
2,42,Dyson Vacuum,Cordless vacuum cleaner with strong suction po...,59,13
